In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from collections import deque
import os
from tqdm import tqdm

In [3]:


# 3.4 Deep Q-Learning Architecture
class DuelingDQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DuelingDQN, self).__init__()
        self.state_dim = state_dim
        self.action_dim = action_dim
        
        # Shared feature layers
        self.fc1 = nn.Linear(state_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        
        # Dueling streams
        self.value_stream = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        
        self.advantage_stream = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim))
    
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        
        value = self.value_stream(x)
        advantage = self.advantage_stream(x)
        
        # Combine value and advantage streams
        q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
        return q_values

# 3.5 Training Protocol
class DQNAgent:
    def __init__(self, state_dim, action_dim, lr=0.00025, gamma=0.99, 
                 epsilon=1.0, epsilon_min=0.05, epsilon_decay=0.995,
                 batch_size=128, memory_size=10000, tau=0.01):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.tau = tau
        
        # Device configuration
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Networks
        self.policy_net = DuelingDQN(state_dim, action_dim).to(self.device)
        self.target_net = DuelingDQN(state_dim, action_dim).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        
        # Optimizer and loss
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=lr)
        self.loss_fn = nn.SmoothL1Loss()  # Huber loss
        
        # Replay memory
        self.memory = deque(maxlen=memory_size)
        
        # Training history
        self.losses = []
        self.rewards = []
        self.utilization_gains = []
        self.policy_errors = []
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_dim)
        else:
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            with torch.no_grad():
                q_values = self.policy_net(state)
            return q_values.argmax().item()
    
    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    
    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        
        # Sample batch from memory
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        
        # Convert to tensors
        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).unsqueeze(1).to(self.device)
        rewards = torch.FloatTensor(rewards).unsqueeze(1).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).unsqueeze(1).to(self.device)
        
        # Compute Q-values
        current_q = self.policy_net(states).gather(1, actions)
        
        # Compute target Q-values
        with torch.no_grad():
            next_q = self.target_net(next_states).max(1)[0].unsqueeze(1)
            target_q = rewards + (1 - dones) * self.gamma * next_q
        
        # Compute loss
        loss = self.loss_fn(current_q, target_q)
        self.losses.append(loss.item())
        
        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()
        
        # Update target network
        for target_param, policy_param in zip(self.target_net.parameters(), self.policy_net.parameters()):
            target_param.data.copy_(self.tau * policy_param.data + (1.0 - self.tau) * target_param.data)
    
    def save_model(self, path):
        torch.save({
            'policy_state_dict': self.policy_net.state_dict(),
            'target_state_dict': self.target_net.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict()
        }, path)
    
    def load_model(self, path):
        checkpoint = torch.load(path)
        self.policy_net.load_state_dict(checkpoint['policy_state_dict'])
        self.target_net.load_state_dict(checkpoint['target_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# 3.6 Environment and Reward Function
class NetworkEnvironment:
    def __init__(self, data):
        self.data = data
        self.current_step = 0
        self.n_steps = len(data)
        
        # Baseline weights from Delphi panel
        self.base_weights = {
            'Teaching': 0.28,
            'Non-teaching': 0.12,
            'Managerial': 0.35,
            'Non-managerial': 0.25
        }
        
        # Initialize weights
        self.current_weights = np.array([
            self.base_weights['Teaching'],
            self.base_weights['Non-teaching'],
            self.base_weights['Managerial'],
            self.base_weights['Non-managerial']
        ])
    
    def reset(self):
        self.current_step = 0
        self.current_weights = np.array([
            self.base_weights['Teaching'],
            self.base_weights['Non-teaching'],
            self.base_weights['Managerial'],
            self.base_weights['Non-managerial']
        ])
        return self._get_state()
    
    def _get_state(self):
        row = self.data.iloc[self.current_step]
        
        # One-hot encode academic phase
        phase = row['phase']
        phase_encoding = {
            'Lecture': [1, 0, 0],
            'Examination': [0, 1, 0],
            'Administrative': [0, 0, 1]
        }
        
        # State vector: [volumes(4), capacity, weights(4), phase(3)]
        state = np.array([
            row['Teaching_volume'],
            row['Non-teaching_volume'],
            row['Managerial_volume'],
            row['Non-managerial_volume'],
            row['capacity'],
            *self.current_weights,
            *phase_encoding[phase]
        ])
        return state
    
    def _apply_action(self, action):
        """Apply action to adjust weights with constraints"""
        # Action mapping: 
        # 0: +10% Teaching, 1: -10% Teaching
        # 2: +10% Non-teaching, 3: -10% Non-teaching
        # 4: +10% Managerial, 5: -10% Managerial
        # 6: +10% Non-managerial, 7: -10% Non-managerial
        
        new_weights = self.current_weights.copy()
        class_idx = action // 2
        direction = 1 if action % 2 == 0 else -1
        
        # Calculate adjustment (10% of base weight)
        adjustment = 0.1 * self.base_weights[list(self.base_weights.keys())[class_idx]] * direction
        
        # Apply adjustment
        new_weights[class_idx] += adjustment
        
        # Maintain group constraints:
        # Academic classes (Teaching + Non-teaching) = 0.4
        # Non-academic classes (Managerial + Non-managerial) = 0.6
        if class_idx in [0, 1]:  # Academic group
            partner_idx = 1 if class_idx == 0 else 0
            new_weights[partner_idx] = 0.4 - new_weights[class_idx]
        else:  # Non-academic group
            partner_idx = 3 if class_idx == 2 else 2
            new_weights[partner_idx] = 0.6 - new_weights[class_idx]
        
        # Clip weights to ±20% of baseline
        for i in range(4):
            base = list(self.base_weights.values())[i]
            min_val = max(0, base * 0.8)
            max_val = min(1, base * 1.2)
            new_weights[i] = np.clip(new_weights[i], min_val, max_val)
        
        return new_weights
    
    def _calculate_reward(self, state, new_weights):
        """Calculate reward based on institutional objectives"""
        # Unpack state
        (teach_vol, nt_vol, mgr_vol, nmgr_vol, 
         capacity, *_, phase1, phase2, phase3) = state
        
        # Calculate demand in Mbps (convert MB to Mbps)
        interval_sec = 5 * 60  # 5-minute intervals
        teach_demand = (teach_vol * 8) / interval_sec
        nt_demand = (nt_vol * 8) / interval_sec
        mgr_demand = (mgr_vol * 8) / interval_sec
        nmgr_demand = (nmgr_vol * 8) / interval_sec
        
        # Calculate allocated bandwidth
        teach_alloc = new_weights[0] * capacity
        nt_alloc = new_weights[1] * capacity
        mgr_alloc = new_weights[2] * capacity
        nmgr_alloc = new_weights[3] * capacity
        
        # Calculate actual throughput
        teach_tput = min(teach_alloc, teach_demand)
        nt_tput = min(nt_alloc, nt_demand)
        mgr_tput = min(mgr_alloc, mgr_demand)
        nmgr_tput = min(nmgr_alloc, nmgr_demand)
        
        # Calculate satisfaction ratios
        teach_sat = min(1, teach_tput / teach_demand) if teach_demand > 0 else 1
        nt_sat = min(1, nt_tput / nt_demand) if nt_demand > 0 else 1
        mgr_sat = min(1, mgr_tput / mgr_demand) if mgr_demand > 0 else 1
        nmgr_sat = min(1, nmgr_tput / nmgr_demand) if nmgr_demand > 0 else 1
        
        # Get policy weights from state (positions 5-8 are weights)
        teach_policy = state[5]
        nt_policy = state[6]
        mgr_policy = state[7]
        nmgr_policy = state[8]
        
        # Calculate utilization
        total_tput = teach_tput + nt_tput + mgr_tput + nmgr_tput
        utilization = total_tput / capacity
        
        # Reward components
        policy_compliance = (
            teach_policy * teach_sat +
            nt_policy * nt_sat +
            mgr_policy * mgr_sat +
            nmgr_policy * nmgr_sat
        )
        
        underutilization_penalty = 0.15 * (capacity - total_tput)
        
        # Critical service assurance (Teaching during exams)
        if state[10] == 1:  # Examination phase
            teach_latency_penalty = 10 * max(0, teach_demand - teach_alloc) / teach_demand if teach_demand > 0 else 0
        else:
            teach_latency_penalty = 0
        
        # Final reward
        reward = policy_compliance - underutilization_penalty - teach_latency_penalty
        
        # Track metrics
        policy_error = np.mean(np.abs([
            teach_policy - new_weights[0],
            nt_policy - new_weights[1],
            mgr_policy - new_weights[2],
            nmgr_policy - new_weights[3]
        ]))
        
        return reward, utilization, policy_error
    
    def step(self, action):
        # Apply action to get new weights
        new_weights = self._apply_action(action)
        
        # Get current state
        current_state = self._get_state()
        
        # Calculate reward based on current state and new weights
        reward, utilization, policy_error = self._calculate_reward(current_state, new_weights)
        
        # Update weights for next state
        self.current_weights = new_weights
        
        # Move to next time step
        self.current_step += 1
        done = self.current_step >= self.n_steps - 1
        
        # Get next state
        next_state = self._get_state() if not done else current_state
        
        return next_state, reward, done, utilization, policy_error

# 3.7 Training Execution
def train_dqn(env, agent, episodes=50, save_interval=5):
    # Create directory for saving models and plots
    os.makedirs("models", exist_ok=True)
    os.makedirs("plots", exist_ok=True)
    
    # Training loop
    for episode in tqdm(range(episodes), desc="Training"):
        state = env.reset()
        total_reward = 0
        episode_utilizations = []
        episode_policy_errors = []
        
        done = False
        while not done:
            action = agent.act(state)
            next_state, reward, done, utilization, policy_error = env.step(action)
            
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            
            total_reward += reward
            episode_utilizations.append(utilization)
            episode_policy_errors.append(policy_error)
            
            agent.replay()
        
        # Update metrics
        agent.rewards.append(total_reward)
        agent.utilization_gains.append(np.mean(episode_utilizations))
        agent.policy_errors.append(np.mean(episode_policy_errors))
        
        # Decay epsilon
        agent.decay_epsilon()
        
        # Save model periodically
        if (episode + 1) % save_interval == 0:
            agent.save_model(f"models/dqn_epoch_{episode+1}.pth")
        
        # Plot progress
        if (episode + 1) % 5 == 0:
            plot_training_progress(agent, episode+1)

# 3.8 Visualization Functions
def plot_training_progress(agent, episode):
    plt.figure(figsize=(18, 12))
    
    # Loss
    plt.subplot(2, 2, 1)
    plt.plot(agent.losses)
    plt.title("Training Loss (Huber)")
    plt.xlabel("Batch Updates")
    plt.ylabel("Loss")
    plt.grid(True)
    
    # Rewards
    plt.subplot(2, 2, 2)
    plt.plot(agent.rewards)
    plt.title("Episode Total Reward")
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.grid(True)
    
    # Utilization
    plt.subplot(2, 2, 3)
    plt.plot(agent.utilization_gains)
    plt.title("Bandwidth Utilization")
    plt.xlabel("Episode")
    plt.ylabel("Utilization Rate")
    plt.ylim(0.5, 1.0)
    plt.grid(True)
    
    # Policy Error
    plt.subplot(2, 2, 4)
    plt.plot(agent.policy_errors)
    plt.title("Policy Weight Deviation")
    plt.xlabel("Episode")
    plt.ylabel("MAE")
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f"plots/training_progress_ep{episode}.png", dpi=300)
    plt.close()

# 3.9 Validation and Testing
def validate_model(env, agent, n_episodes=10):
    test_rewards = []
    test_utilizations = []
    test_policy_errors = []
    
    for _ in tqdm(range(n_episodes), desc="Validation"):
        state = env.reset()
        total_reward = 0
        episode_utilizations = []
        episode_policy_errors = []
        
        done = False
        while not done:
            action = agent.act(state)
            next_state, reward, done, utilization, policy_error = env.step(action)
            
            state = next_state
            total_reward += reward
            episode_utilizations.append(utilization)
            episode_policy_errors.append(policy_error)
        
        test_rewards.append(total_reward)
        test_utilizations.append(np.mean(episode_utilizations))
        test_policy_errors.append(np.mean(episode_policy_errors))
    
    # Plot validation results
    plt.figure(figsize=(15, 10))
    
    plt.subplot(1, 3, 1)
    plt.bar(range(n_episodes), test_rewards)
    plt.title("Validation Rewards")
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    
    plt.subplot(1, 3, 2)
    plt.bar(range(n_episodes), test_utilizations)
    plt.title("Bandwidth Utilization")
    plt.xlabel("Episode")
    plt.ylabel("Utilization Rate")
    plt.ylim(0.6, 0.95)
    
    plt.subplot(1, 3, 3)
    plt.bar(range(n_episodes), test_policy_errors)
    plt.title("Policy Weight Deviation")
    plt.xlabel("Episode")
    plt.ylabel("MAE")
    
    plt.tight_layout()
    plt.savefig("plots/validation_results.png", dpi=300)
    plt.close()
    
    return {
        'mean_reward': np.mean(test_rewards),
        'mean_utilization': np.mean(test_utilizations),
        'mean_policy_error': np.mean(test_policy_errors)
    }

# Main execution
if __name__ == "__main__":
    # Load simulated data
    data = pd.read_csv("network_traffic_simulation.csv")
    
    # Initialize environment and agent
    env = NetworkEnvironment(data)
    state_dim = 12  # 4 volumes + 1 capacity + 4 weights + 3 phase indicators
    action_dim = 8  # 8 discrete actions
    agent = DQNAgent(state_dim, action_dim)
    
    # Train the model
    train_dqn(env, agent, episodes=50)
    
    # Save final model
    agent.save_model("models/dqn_final.pth")
    
    # Validation
    val_results = validate_model(env, agent)
    print("\nValidation Results:")
    print(f"Mean Reward: {val_results['mean_reward']:.2f}")
    print(f"Mean Utilization: {val_results['mean_utilization']:.2%}")
    print(f"Mean Policy Error: {val_results['mean_policy_error']:.4f}")
    
    # Plot final training progress
    plot_training_progress(agent, "final")

Training:   0%|                                                                                 | 0/50 [00:00<?, ?it/s]C:\Users\markov-consult\AppData\Local\Temp\ipykernel_4880\3730523736.py:95: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  states = torch.FloatTensor(states).to(self.device)
Validation: 100%|██████████████████████████████████████████████████████████████████████| 10/10 [02:25<00:00, 14.53s/it]



Validation Results:
Mean Reward: -143492.18
Mean Utilization: 58.19%
Mean Policy Error: 0.0087
